In [1]:
import pandas as pd
from pulp import *

### Read the given nutrition dataset into a Pandas DataFrame object
Note we are reading only the first 64 rows with `nrows=64` argument because we just want to read all the nutrients informtion and not the maximum/minimum bounds in the dataset. We will enter those bounds in the optimization problem separately.

In [2]:
df = pd.read_excel("/Users/kailashthiyagarajan/Downloads/local/code/Optimization/Optimization-Python/Data/diet - medium.xls",nrows=17)

### Show the dataset

In [3]:
df

,Foods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Sodium (mg),Carbohydrates (g),Dietary_Fiber (g),Protein (g),Vit_A (IU),Vit_C (IU),Calcium (mg),Iron (mg)
0,Frozen Broccoli,0.48,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,Frozen Corn,0.54,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
2,Raw Lettuce Iceberg,0.06,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
3,Baked Potatoes,0.18,1/2 Cup,171.5,0.0,0.2,15.2,39.9,3.2,3.7,0.0,15.6,22.7,4.3
4,Tofu,0.93,1/4 block,88.2,0.0,5.5,8.1,2.2,1.4,9.4,98.6,0.1,121.8,6.2
5,Roasted Chicken,2.52,1 lb chicken,277.4,129.9,10.8,125.6,0.0,0.0,42.2,77.4,0.0,21.9,1.8
6,Spaghetti W/ Sauce,2.34,1 1/2 Cup,358.2,0.0,12.3,1237.1,58.3,11.6,8.2,3055.2,27.9,80.2,2.3
7,Raw Apple,0.72,"1 Fruit,3/Lb,Wo/Rf",81.4,0.0,0.5,0.0,21.0,3.7,0.3,73.1,7.9,9.7,0.2
8,Banana,0.45,"1 Fruit,Wo/Skn&Seeds",104.9,0.0,0.5,1.1,26.7,2.7,1.2,92.3,10.4,6.8,0.4
9,Wheat Bread,0.15,1 Sl,65.0,0.0,1.0,134.5,12.4,1.3,2.2,0.0,0.0,10.8,0.7


### Create the `PuLP` problem variable. Since it is a cost minimization problem, we need to use `LpMinimize`

In [4]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Simple Diet Problem",LpMinimize)

/Users/kailashthiyagarajan/opt/anaconda3/envs/opt_env/lib/python3.7/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Create a list of food items from the dataset

In [5]:
# Creates a list of the Ingredients
food_items = list(df['Foods'])

In [6]:
print("So, the food items to consdier, are\n"+"-"*100)
for f in food_items:
    print(f,end=', ')

So, the food items to consdier, are
----------------------------------------------------------------------------------------------------
Frozen Broccoli, Frozen Corn, Raw Lettuce Iceberg,  Baked Potatoes, Tofu, Roasted Chicken, Spaghetti W/ Sauce, Raw Apple, Banana, Wheat Bread, White Bread, Oatmeal Cookies, Apple Pie, Scrambled Eggs, Turkey Bologna, Beef Frankfurter, Chocolate Chip Cookies, 

### Create a dictinary of costs for all food items

In [7]:
costs = dict(zip(food_items,df['Price/Serving']))

In [8]:
costs

{'Frozen Broccoli': 0.48,
 'Frozen Corn': 0.54,
 'Raw Lettuce Iceberg': 0.06,
 ' Baked Potatoes': 0.18,
 'Tofu': 0.9299999999999999,
 'Roasted Chicken': 2.52,
 'Spaghetti W/ Sauce': 2.34,
 'Raw Apple': 0.72,
 'Banana': 0.44999999999999996,
 'Wheat Bread': 0.15000000000000002,
 'White Bread': 0.18,
 'Oatmeal Cookies': 0.27,
 'Apple Pie': 0.48,
 'Scrambled Eggs': 0.33,
 'Turkey Bologna': 0.44999999999999996,
 'Beef Frankfurter': 0.81,
 'Chocolate Chip Cookies': 0.09}

### Create a dictionary of calories for all food items

In [9]:
calories = dict(zip(food_items,df['Calories']))

### Create a dictionary of cholesterol for all food items

In [10]:
cholesterol = dict(zip(food_items,df['Cholesterol (mg)']))

### Create a dictionary of total fat for all food items

In [11]:
fat = dict(zip(food_items,df['Total_Fat (g)']))

### Create a dictionary of sodium for all food items

In [12]:
sodium = dict(zip(food_items,df['Sodium (mg)']))

### Create a dictionary of carbohydrates for all food items

In [13]:
carbs = dict(zip(food_items,df['Carbohydrates (g)']))

### Create a dictionary of dietary fiber for all food items

In [14]:
fiber = dict(zip(food_items,df['Dietary_Fiber (g)']))

### Create a dictionary of protein for all food items

In [15]:
protein = dict(zip(food_items,df['Protein (g)']))

### Create a dictionary of vitamin A for all food items

In [16]:
vit_A = dict(zip(food_items,df['Vit_A (IU)']))

### Create a dictionary of vitamin C for all food items

In [17]:
vit_C = dict(zip(food_items,df['Vit_C (IU)']))

### Create a dictionary of calcium for all food items

In [18]:
calcium = dict(zip(food_items,df['Calcium (mg)']))

### Create a dictionary of iron for all food items

In [19]:
iron = dict(zip(food_items,df['Iron (mg)']))

### Create a dictionary of food items with lower bound

In [20]:
# A dictionary called 'food_vars' is created to contain the referenced Variables
food_vars = LpVariable.dicts("Food",food_items,0,cat='Continuous')

In [21]:
food_vars

{'Frozen Broccoli': Food_Frozen_Broccoli,
 'Frozen Corn': Food_Frozen_Corn,
 'Raw Lettuce Iceberg': Food_Raw_Lettuce_Iceberg,
 ' Baked Potatoes': Food__Baked_Potatoes,
 'Tofu': Food_Tofu,
 'Roasted Chicken': Food_Roasted_Chicken,
 'Spaghetti W/ Sauce': Food_Spaghetti_W__Sauce,
 'Raw Apple': Food_Raw_Apple,
 'Banana': Food_Banana,
 'Wheat Bread': Food_Wheat_Bread,
 'White Bread': Food_White_Bread,
 'Oatmeal Cookies': Food_Oatmeal_Cookies,
 'Apple Pie': Food_Apple_Pie,
 'Scrambled Eggs': Food_Scrambled_Eggs,
 'Turkey Bologna': Food_Turkey_Bologna,
 'Beef Frankfurter': Food_Beef_Frankfurter,
 'Chocolate Chip Cookies': Food_Chocolate_Chip_Cookies}

### Adding the objective function to the problem

In [22]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

### Adding the calorie constraints to the problem

In [23]:
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0, "CalorieMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1300.0, "CalorieMaximum"

### Adding other nutrient constraints to the problem one by one...

# Cholesterol
prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) >= 30.0, "CholesterolMinimum"
prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) <= 240.0, "CholesterolMaximum"

# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 40.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 100.0, "FatMaximum"

# Sodium
prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) >= 500.0, "SodiumMinimum"
prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) <= 2000.0, "SodiumMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 450.0, "CarbsMaximum"

# Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 125.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 250.0, "FiberMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 60.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 100.0, "ProteinMaximum"

# Vitamin A
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) >= 1000.0, "VitaminAMinimum"
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) <= 10000.0, "VitaminAMaximum"

# Vitamin C
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) >= 400.0, "VitaminCMinimum"
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) <= 5000.0, "VitaminCMaximum"

# Calcium
prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) >= 300.0, "CalciumMinimum"
prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) <= 1500.0, "CalciumMaximum"

# Iron
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) >= 10.0, "IronMinimum"
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) <= 40.0, "IronMaximum"

In [24]:
# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0, "FatMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0, "CarbsMaximum"

# Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 60.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 125.0, "FiberMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 100.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 150.0, "ProteinMaximum"

In [25]:
prob

Simple_Diet_Problem:
MINIMIZE
0.48*Food_Apple_Pie + 0.44999999999999996*Food_Banana + 0.81*Food_Beef_Frankfurter + 0.09*Food_Chocolate_Chip_Cookies + 0.48*Food_Frozen_Broccoli + 0.54*Food_Frozen_Corn + 0.27*Food_Oatmeal_Cookies + 0.72*Food_Raw_Apple + 0.06*Food_Raw_Lettuce_Iceberg + 2.52*Food_Roasted_Chicken + 0.33*Food_Scrambled_Eggs + 2.34*Food_Spaghetti_W__Sauce + 0.9299999999999999*Food_Tofu + 0.44999999999999996*Food_Turkey_Bologna + 0.15000000000000002*Food_Wheat_Bread + 0.18*Food_White_Bread + 0.18*Food__Baked_Potatoes + 0.0
SUBJECT TO
CalorieMinimum: 67.2 Food_Apple_Pie + 104.9 Food_Banana
 + 141.8 Food_Beef_Frankfurter + 78.1 Food_Chocolate_Chip_Cookies
 + 73.8 Food_Frozen_Broccoli + 72.2 Food_Frozen_Corn + 81 Food_Oatmeal_Cookies
 + 81.4 Food_Raw_Apple + 2.6 Food_Raw_Lettuce_Iceberg
 + 277.4 Food_Roasted_Chicken + 99.6 Food_Scrambled_Eggs
 + 358.2 Food_Spaghetti_W__Sauce + 88.2 Food_Tofu + 56.4 Food_Turkey_Bologna
 + 65 Food_Wheat_Bread + 65 Food_White_Bread + 171.5 Food__Bak

### Writing problem data to a `.lp` file

In [26]:
# The problem data is written to an .lp file
prob.writeLP("SimpleDietProblem.lp")

[Food_Apple_Pie,
 Food_Banana,
 Food_Beef_Frankfurter,
 Food_Chocolate_Chip_Cookies,
 Food_Frozen_Broccoli,
 Food_Frozen_Corn,
 Food_Oatmeal_Cookies,
 Food_Raw_Apple,
 Food_Raw_Lettuce_Iceberg,
 Food_Roasted_Chicken,
 Food_Scrambled_Eggs,
 Food_Spaghetti_W__Sauce,
 Food_Tofu,
 Food_Turkey_Bologna,
 Food_Wheat_Bread,
 Food_White_Bread,
 Food__Baked_Potatoes]

### Run the solver

In [27]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kailashthiyagarajan/opt/anaconda3/envs/opt_env/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/wl/hpzsvb551856hqjkw0rzm8s00000gn/T/338ac08528f44070a180f26ea5fc1a79-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/wl/hpzsvb551856hqjkw0rzm8s00000gn/T/338ac08528f44070a180f26ea5fc1a79-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 189 RHS
At line 200 BOUNDS
At line 201 ENDATA
Problem MODEL has 10 rows, 17 columns and 156 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 10 (0) rows, 17 (0) columns and 156 (0) elements
0  Obj 0 Primal inf 13.567816 (5)
8  Obj 5.5182293
Optimal - objective value 5.5182293
Optimal objective 5.518229284 - 8 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds

1

### Print the problem solution status `'optimal'`, `'infeasible'`, `'unbounded'` etc...

In [28]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


### Scan through the problem variables and print out only if the variable quanity is positive i.e. it is included in the optimal solution

In [29]:
print("Therefore, the optimal (least cost) balanced diet consists of\n"+"-"*110)
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Therefore, the optimal (least cost) balanced diet consists of
--------------------------------------------------------------------------------------------------------------
Food_Frozen_Broccoli = 6.9242113
Food_Scrambled_Eggs = 6.060891
Food__Baked_Potatoes = 1.0806324


### Print the optimal diet cost

In [30]:
print("The total cost of this balanced diet is: ${}".format(round(value(prob.objective),2)))

The total cost of this balanced diet is: $5.52
